This notebook downloads summoner IDs for a given rank, division, and region.

In [45]:
# IMPORTS
import requests
import numpy as np
import time
import pandas as pd
import os
import datetime
from io import TextIOWrapper
from fastparquet import write
from pandas import DataFrame

In [38]:
API_KEY = os.getenv('RIOT_API_KEY')
API_KEY = 'RGAPI-295e93e6-ce21-4551-89e3-dfc11ebb5b14'

In [42]:
def get_summoner_ids(region: str, rank: str, division: str, num_pages=4) -> DataFrame:
    """
    Gets random summoner ids from North America from the various ranks (Diamond:Platinum:Gold:Silver:Bronze)
    Returns a pandas dictionary only with summoner ids that have wins + losses >= 20
    Args:
        region (str): The region we want to search in
        rank (str): The rank we want to get from
        division (str): The divions we want to get from
        num_pages (int, optional): An amount of summoner ids to get, a group of 204 are a 'page'. Defaults to 4.

    Returns:
        bool: Whether getting the summoner ids was successful
    """
    print(f"Current rank {rank} {division}")
    df = pd.DataFrame(columns=['leagueId',
                               'queueType',
                               'tier',
                               'rank',
                               'summonerId',
                               'summonerName',
                               'leaguePoints',
                               'wins',
                               'losses',
                               'veteran',
                               'inactive',
                               'freshBlood',
                               'hotStreak'])
    for page_num in range(1, num_pages+1):
        URL = f'https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}/?page={page_num}&api_key={API_KEY}'
        response = requests.get(URL)
        for summoner in response.json():
            # We only take people with greater than or equal to 20 combined wins and losses
            if summoner['wins'] + summoner['losses'] >= 20:
               df = pd.concat([df, pd.DataFrame.from_records(summoner, index=[0])], ignore_index=True)
    return df

In [43]:
df = get_summoner_ids('na1', 'DIAMOND', 'II', 1)

Current rank DIAMOND II
205


In [57]:
def write_to_parquet(df: DataFrame) -> str:
    DIRECTORY_PATH = '/Users/ethanshapiro/Repository/Esports Scripts/MOBA Recommender and Prediction/data/raw_data/'
    date_str = str(datetime.datetime.today()).split()[0]
    file_name = f'{date_str}_{df.loc[0, "tier"]}_{df.loc[0, "rank"]}.parquet'
    file_path = DIRECTORY_PATH + file_name
    if not os.path.isfile(file_path):
        write(file_path, df)
    else:
        write(file_path, df, append=True)
    return file_path

In [59]:
new_file = write_to_parquet(df)

In [ ]:
regions = ['na1', 'kr', 'euw1']
ranks = ['DIAMOND', 'PLATINUM']
tiers = ['I', 'II', 'III', 'IV']